In [1]:
import torch

/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import seq2seq.optim

In [3]:
!ls

autoencoder.py		checkpoint.py  __pycache__	temp.py
autoencoder_test.ipynb	models	       pytorch-seq2seq	test.py
batch_predictor.py	predictor.py   seq2seq


In [4]:
import os
import argparse
import logging
import timeit
import torch

#from seq2seq.evaluator import BatchPredictor
#from seq2seq.util.checkpoint import Checkpoint

from batch_predictor import BatchPredictor
from checkpoint import Checkpoint

# Load models directly because the package is installed
#
#checkpoint_path = "models/2020_04_22_20_36_50" # 91% accuracy model, only core theories
checkpoint_path = "models/2020_04_26_20_11_28" # 95% accuracy model, core theories + integer + sorting
#checkpoint_path = "models/2020_09_24_23_38_06" # 98% accuracy model, core theories + integer + sorting | separate theory tokens
# checkpoint_path = "autoencoder/models/2020_11_28_16_45_10" # 96-98% accuracy model, core theories + integer + sorting + real | separate theory tokens

#checkpoint_path = "models/2020_12_04_03_47_22" # 97% accuracy model, core theories + integer + sorting + real + bag | separate theory tokens

################## 
#checkpoint_path = "models/2021_02_21_15_46_04" # 98% accuracy model, up to probability theory

#heckpoint_path = "models/2021_02_22_16_07_03" # 97-98% accuracy model, up to and include probability theory

# logging.info("loading checkpoint from {}".format(checkpoint_path))

checkpoint = Checkpoint.load(checkpoint_path)
seq2seq = checkpoint.model
input_vocab = checkpoint.input_vocab
output_vocab = checkpoint.output_vocab

batch_encoder = BatchPredictor(seq2seq, input_vocab, output_vocab)

predictor = BatchPredictor(seq2seq, input_vocab, output_vocab)

seqs = ['@ C$bool$ ! | Vs @ @ C$min$ = @ @ C$pred_set$ SUBSET C$pred_set$ UNIV Vs @ @ C$min$ = Vs C$pred_set$ UNIV']#["@ @ Cmin$= CConseqConv$ASM_MARKER | Vy | Vx Vx", "@ @ Cmin$= CConseqConv$ASM_MARKER | Vy | Vy Vt", "@ Cbool$! | Vh1 @ Cbool$! | Vh2 @ @ Cmin$==> @ Cbool$~ @ @ Cmin$= Vh1 Vh2 @ Cbool$! | Vl1 @ Cbool$! | Vl2 @ Cbool$~ @ @ Cmin$= @ @ Clist$CONS Vh1 Vl1 @ @ Clist$CONS Vh2 Vl2","@ Cbool$! | Vh1 @ Cbool$! | Vh2 @ @ Cmin$==> @ Cbool$~ @ @ Cmin$= Vh1 Vh2 @ Cbool$! | Vl1 @ Cbool$! | Vl2 @ Cbool$~ @ @ Cmin$= @ @ Clist$CONS Vh1 Vl1 @ @ Clist$CONS Vh2 Vl2","@ Cbool$! | Vh1 @ Cbool$! | Vh2 @ @ Cmin$==> @ Cbool$~ @ @ Cmin$= Vh1 Vh2 @ Cbool$! | Vl1 @ Cbool$! | Vl2 @ Cbool$~ @ @ Cmin$= @ @ Clist$CONS Vh1 Vl1 @ @ Clist$CONS Vh2 Vl2"]

#seqs = ["@ Cbool$~ @ @ @ Clist$SHORTLEX VR Vl Clist$NIL"]
seqs = [['@', '@', 'Cmin$=', '@', '@', 'Cmin$=', '@', 'Clist$REVERSE', 'Vl', 'Clist$NIL', '@', '@', 'Cmin$=', 'Vl', 'Clist$NIL']]#, 
# ['@', '@', 'Cmin$=', '@', '@', 'Cmin$=', '@', 'Clist$REVERSE', 'Clist$NIL', 'Clist$NIL', '@', '@', 'Cmin$=', 'Clist$NIL', 'Clist$NIL'], 
 #['@', '@', 'Dmin$==>', '@', '@', 'Cmin$=', '@', '@', 'Cmin$=', '@', 'Clist$REVERSE', 'Vl', 'Clist$NIL', '@', '@', 'Cmin$=', 'Vl', 'Clist$NIL', '@', 'Cbool$!', '|', 'Vh', '@', '@', 'Cmin$=', '@', '@', 'Cmin$=', '@', 'Clist$REVERSE', '@', '@', 'Clist$CONS', 'Vh', 'Vl', 'Clist$NIL', '@', '@', 'Cmin$=', '@', '@', 'Clist$CONS', 'Vh', 'Vl', 'Clist$NIL'], ['@', '@', 'Dmin$==>', '@', '@', 'Cmin$=', '@', '@', 'Cmin$=', '@', 'Clist$REVERSE', 'Vl', 'Clist$NIL', '@', '@', 'Cmin$=', 'Vl', 'Clist$NIL', '@', 'Cbool$!', '|', 'Vh', '@', '@', 'Cmin$=', '@', '@', 'Cmin$=', '@', 'Clist$REVERSE', '@', '@', 'Clist$CONS', 'Vh', 'Vl', 'Clist$NIL', '@', '@', 'Cmin$=', '@', '@', 'Clist$CONS', 'Vh', 'Vl', 'Clist$NIL']]



#seqs = [i.strip().split() for i in seqs]

# relatively fast encoder: 2~5ms a batch
# compared to HOL's 0.1s time limit of tactic execution, encoding is not a bottleneck now

print(batch_encoder.predict(seqs))

s1 = timeit.default_timer()

# size: (2 [by default bidirectional], batch_num, hidden_length)
out, sizes = batch_encoder.encode(seqs)
representation = torch.cat(out.split(1), dim=2).squeeze()
print(representation)
print(representation.shape)
print(out)

s2 = timeit.default_timer()
print(s2-s1)


#s1 = timeit.default_timer()
# out, sizes = batch_encoder.encode(["@ Cbool$! | Vh1 @ Cbool$! | Vh2 @ @ Cmin$==> @ Cbool$~ @ @ Cmin$= Vh1 Vh2 @ Cbool$! | Vl1 @ Cbool$! | Vl2 @ Cbool$~ @ @ Cmin$= @ @ Clist$CONS Vh1 Vl1 @ @ Clist$CONS Vh2 Vl2"])

# out, sizes = batch_encoder.encode(["@ Cbool$! | Vh1 @ Cbool$! | Vh2 @ @ Cmin$==> @ Cbool$~ @ @ Cmin$= Vh1 Vh2 @ Cbool$! | Vl1 @ Cbool$! | Vl2 @ Cbool$~ @ @ Cmin$= @ @ Clist$CONS Vh1 Vl1 @ @ Clist$CONS Vh2 Vl2".strip().split()])

# # merge two hidden variables
# representations = torch.cat(out.split(1), dim=2).squeeze(0)
# print(representations)
# # print(representations.shape)
# s2 = timeit.default_timer()    
# print(s2-s1)

[['@', '@', 'Cmin$=', '@', '@', 'Cmin$=', '@', 'Clist$REVERSE', 'Vl', 'Clist$NIL', '@', '@', 'Cmin$=', 'Vl', 'Clist$NIL', '<eos>']]
tensor([ 0.9227, -0.9994,  0.9012, -0.1492, -0.7326, -0.1064,  0.9871, -0.3356,
        -0.2293, -0.1615,  0.9423, -0.9502, -0.4562,  0.9700,  0.3955,  0.1864,
         0.2855, -0.7346,  0.6362, -0.3392,  0.9620, -0.3084,  0.7971,  0.6478,
         0.7215, -0.8193, -0.1357, -0.2715,  0.7896, -0.4495,  0.7842, -0.8479,
         0.5934, -0.8262, -0.0249,  0.1982,  0.5330, -0.9736,  0.2923,  0.1848,
        -0.2433, -0.6897,  0.9870, -0.9894, -0.8268,  0.9557, -0.2367, -0.3137,
         0.3133, -0.3285,  0.1337,  0.7346,  0.6674, -0.7748,  0.9517, -0.3573,
         0.1711, -0.9966,  0.6222,  0.9682,  0.0441, -0.0091, -0.6403, -0.4003,
        -0.6224, -0.1272, -0.1120, -0.7380,  0.9291,  0.9387,  0.2281,  0.5093,
        -0.9825,  0.2802,  0.0792, -0.8104,  0.9210, -0.8724, -0.0052,  0.6844,
         0.2957,  0.2354,  0.1477,  0.9221, -0.8610, -0.7140,  0.361

/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'seq2seq.models.EncoderRNN.EncoderRNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the origina

In [5]:
# seq2seq.decoder.use_attention = False
# dec = seq2seq.decoder(None, out, representations)[2]# representations))
